In [1]:
# Imports
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import SVD, SVDpp, NMF
from surprise import SlopeOne, CoClustering
from surprise.prediction_algorithms import predictions

%load_ext autoreload
%autoreload 2

In [2]:
from helpersSlopeOne import*

# Load dataset and samples into a pandas data frame
DATA_TRAIN_PATH = 'data/data_train.csv'
data_np = load_data(DATA_TRAIN_PATH)


DATA_TEST_PATH = 'data/sampleSubmission.csv'
samples = load_data(DATA_TEST_PATH)

In [3]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data_np[['user_id', 'movie_id', 'rating']], reader=reader)

## Basic Slope One
ID : 25592
- RMSE : 1.040
- Secondary: 0.134

In [6]:
slope_cv = cross_validate(SlopeOne(), data, cv=5, n_jobs=5, verbose=False)

In [11]:
print('SlopeOne', '\t', round(slope_cv['test_rmse'].mean(), 4), '\t', round(slope_cv['test_mae'].mean(), 4))

SlopeOne 	 1.0001 	 0.8017


In [4]:
trainset = data.build_full_trainset()
algo = SlopeOne().fit(trainset)

In [5]:
samples['rating'] = samples[['user_id', 'movie_id']] \
.apply(lambda row: algo.predict(row['user_id'], row['movie_id'])[3], axis=1)

samples.head()

,user_id,movie_id,rating
0,37,1,3.245664
1,73,1,2.953050
2,156,1,3.735032
3,160,1,3.300362
4,248,1,3.317072


In [14]:
samples['rating'] = samples['rating'].apply(round)

In [6]:
PATHOUT = "Predictions.csv"
create_csv(PATHOUT,samples)